**Objective : Processing Document using RAG-FAISS**

# Set up Environment

In [2]:
import os
os.environ["OPENAI_API_KEY"]="ENTER_OPENAI_API_KEY_HERE"


This function essentially loads and prepares a language model for further use in a larger application

In [3]:
def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1) #gpt-4o-mini
        return llm

# Creating a ChatPromptTemplate

The function creates and returns a custom chat prompt template for use in Langchain.

In [4]:
from langchain.prompts import ChatPromptTemplate

def load_prompt():
        prompt = """ You need to answer the question in the sentence as same as in the  pdf content. . 
        Given below is the context and question of the user.
        context = {context}
        question = {question}
        if the answer is not in the pdf , answer "Please change the question!"
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt

# Import Dependencies

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


def load_knowledgeBasee():
        embeddings=OpenAIEmbeddings()
        DB_FAISS_PATH = 'vectorstore/db_faiss1'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings,allow_dangerous_deserialization=True)
        return db

In [6]:
knowledgeBase=load_knowledgeBasee()
llm=load_llm()
prompt=load_prompt()

In [7]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

# Performing Retrieval Augumented Generation pipeline with multiple queries

This code sets up a retrieval-augmented generation (RAG) pipeline where similar documents are retrieved based on the query, a language model generates answers based on the retrieved context, and the final responses are returned for each query

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
query = "who is krishna"
query4 = "who is Duryodhana"
query5 = "who is kunti"
if True:
    similar_embeddings=knowledgeBase.similarity_search(query)
    similar_embeddings=knowledgeBase.similarity_search(query4)
    similar_embeddings=knowledgeBase.similarity_search(query5)
    similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings())
                
    #creating the chain for integrating llm,prompt,stroutputparser
    retriever = similar_embeddings.as_retriever()
    rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )
                
    response4=rag_chain.invoke(query4)
    response = rag_chain.invoke(query)
    response5=rag_chain.invoke(query5)

In [9]:
response5

'Kuntî – Mother of the På√∂avas and aunt of Çrî K®ß√a.'

In [10]:
response

'K®ß√a appeared in V®ndåvana, India, 5,237 years ago and performed His pastimes of love with His devotees and with those who had reached perfection through bhakti-yoga.'

In [11]:
response4

'Duryodhana is the eldest son of King Dhritarashtra and the main antagonist in the Mahabharata, who fought against the Pandavas in the great war at Kurukshetra.'

In [12]:
query3 = "who is krsna"
response3 = rag_chain.invoke(query3)
response3

'Krsna appeared in Vrindavana, India, 5,237 years ago and performed His pastimes of love with His devotees and with those who had reached perfection through bhakti-yoga.'

In [13]:
query2 = "who is Arjuna"
response2 = rag_chain.invoke(query2)
response2

"Arjuna is the disciple in the sacred conversation between Çrî K®ß√a and himself, where he agrees to follow K®ß√a's instructions after his doubts and delusions are dispelled."

In [16]:
query6 = "who is the elder brother of krsna"
response6 = rag_chain.invoke(query6)
response6

'Balarama'

In [17]:
query7 = "who is the father of Abhimanyu "
response7 = rag_chain.invoke(query7)
response7

'The father of Abhimanyu is Arjuna.'

In [21]:
query8 = "who was vidhura and how he is related to vedvyasa"
response8 = rag_chain.invoke(query8)
response8

'Vidura was the half-brother of King Dhritarashtra and Pandu. He was born from the union of Veda Vyasa and a servant maid. He is considered as one of the wisest characters in the Mahabharata.'

In [42]:
query9 = "who used brahmastra on uttara's womb ?"
response9 = rag_chain.invoke(query9)
response9

"Arjuna used the brahmastra on Uttara's womb."